# Hausaufgabe 3
## Particle Image Velocimetry
<!-- Lizensiert unter (CC BY 2.0) Gert Herold, 2020 -->

Zur Charakterisierung der Bewegung in einem Strömungsfeldes findet häufig das optische Messverfahren [Particle Image Velocimetry (PIV)](https://de.wikipedia.org/wiki/Particle_Image_Velocimetry) Anwendung.
Hierfür werden in das Fluid eingebrachte Partikel mit einem zur Ebene aufgeweiteten Laserstrahl beleuchtet und in kurzen Abständen nacheinander fotografiert.
Aus der Ortsveränderung der Partikel und dem bekannten Abstand zwischen den Aufnahmen können die Geschwindigkeitsvektoren bestimmt werden.

Es sollen Funktionen geschrieben werden, mit deren Hilfe sich Berechnung und Visualisierung eines Geschwindigkeits-Vektorfeldes basierend auf zwei aufgenommenen Bildern realisieren lassen.

Zunächst werden einige Module importiert, die hilfreiche Funktionen zur Verfügung stellen:

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

**1) Schreiben Sie eine Funktion *loaddata()* die zwei Arrays mit (Bildhöhe) $\times$ (Bildbreite) Einträgen zurückgibt, die die Bildinformationen (Helligkeitswerte als Zahl) enthalten.**

  * Als Parameter sollen hierfür zwei Dateipfade übergeben werden. Das Einlesen kann z.B. mittels [imread](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.imread.html) geschehen.
  * Eventuell im Bild enthaltene Farbinformationen sind hier nicht weiter von Interesse. Sind im Bild mehrere Farbkanäle pro Pixel enthalten (z.B. RGB), soll die Information auf einen Kanal reduziert werden.
  * Es soll direkt überprüft werden, ob beide Bilder dieselben Abmaße haben. Andernfalls ist eine Fehlermeldung auszugeben: 
  ```python 
        raise ValueError('Dimensions of input images do not match!')
  ```

In [3]:
# Hier eigenen Code schreiben ...
def loaddata(path1,path2):
    im1 = plt.imread(path1)
    im2 = plt.imread(path2)
    if im1.shape != im2.shape:
        raise ValueError('Dimensions of input images do not match!')
    else:
        if len(im1.shape) > 2:
            if im1.shape[2] == 2:
                im1 = np.mean(im1[:,:,:3],2)
            else:
                im1 = np.mean(im1[:,:,:4],2)
        if len(im2.shape) > 2:
            if im2.shape[2] == 2:
                im2 = np.mean(im2[:,:,:3],2)
            else:
                im2 = np.mean(im2[:,:,:4],2)
        return im1, im2

**2) Testen Sie das Laden der Dateien. Die verwendeten Dateien finden Sie im separaten Archiv "*img_HA3.zip*".**

  * **Geben Sie die Dimensionen der zurückgegebenen Arrays an, wenn Sie als Input die folgenden Dateipaare verwenden:**

|Bild 1| Bild 2|
|-|-|
|[B005_1.tif](http://www.pivchallenge.org/pub/index.html#b)|[B005_2.tif](http://www.pivchallenge.org/pub/index.html#b)|
|[B038a.bmp](http://www.pivchallenge.org/pub03/index.html#b)|[B038b.bmp](http://www.pivchallenge.org/pub03/index.html#b)|
|[B_010.TIF](http://www.pivchallenge.org/pub05/index.html#b)|[B_014.TIF](http://www.pivchallenge.org/pub05/index.html#b)|
|[A001_1.tif](http://www.pivchallenge.org/pub/index.html#a)|[A001_2.tif](http://www.pivchallenge.org/pub/index.html#a)|
|[A045a.tif](http://www.pivchallenge.org/pub03/index.html#a)|[A045b.tif](http://www.pivchallenge.org/pub03/index.html#a)|

  * **Überprüfen Sie das Verhalten Ihrer Funktion, wenn folgendes Dateipaar geladen wird:**
  
|Bild 1| Bild 2|
|-|-|
|[B038a.bmp](http://www.pivchallenge.org/pub03/index.html#b)|[A001_1.tif](http://www.pivchallenge.org/pub05/index.html#c)|

Die Bilddateien sind Testfälle der "PIV Challenge"-Benchmark-Initiative (http://www.pivchallenge.org/). Die Quellen der jeweiligen Bilder sind in der Tabelle verlinkt.

In [4]:
#Hier eigenen Code schreiben ...
prefix = 'img/'
path_list = [['B005_1.tif', 'B005_2.tif'], ['B038a.bmp', 'B038b.bmp'], ['B_010.TIF', 'B_010.TIF'], ['A001_1.tif', 'A001_2.tif'], ['A045a.tif', 'A045b.tif'],['B038a.bmp','A001_1.tif']]
for path in path_list:
    im1, im2 = loaddata(prefix + path[0], prefix + path[1])
    print(path[0], path[1])
    print(im1.shape, im2.shape)

B005_1.tif B005_2.tif
(512, 512) (512, 512)
B038a.bmp B038b.bmp
(512, 1536) (512, 1536)
B_010.TIF B_010.TIF
(688, 1440) (688, 1440)
A001_1.tif A001_2.tif
(1024, 1280) (1024, 1280)
A045a.tif A045b.tif
(1004, 992) (1004, 992)


ValueError: Dimensions of input images do not match!

 * **Laden Sie die Daten aus dem Dateipaar (`B005_1.tif`, `B005_2.tif`) und visualisieren Sie den jeweiligen Array-Inhalt in zwei benachbarten Plots.**

In [5]:
# Hier eigenen Code schreiben ...
im1, im2 = loaddata(prefix + path_list[0][0], prefix + path_list[0][1])
plt.subplot(1,2,1)
plt.imshow(im1)
plt.subplot(1,2,2)
plt.imshow(im2)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**3) Schreiben Sie eine Funktion *piv()*, die zwei Dateipfade übergeben bekommt und basierend auf den Bilddaten ein Geschwindigkeitsvektorfeld berechnet.**

  * **Zusätzlich zu den beiden Dateipfaden sollen optional mindestens folgende Parameter übergeben werden können:**
    * *size_interr_window*: Seitenlänge (in Pixel) des quadratischen Ausschnitts, dessen Bewegung untersucht werden soll. Standardwert: $20$
    * *size_search_window*: Seitenlänge (in Pixel) des quadratischen Ausschnitts, innerhalb dessen die höchste Korrelation gesucht werden soll. Der Wert muss sinnvollerweise mindestens so groß wie *size_interr_window* sein (andernfalls Fehlermeldung ausgeben). Standardwert: *None* (setzen auf *size_interr_window*, falls nicht angegeben)
  * **Folgende Werte sollen zurückgegeben werden:**
    * *X*, *Y*: jeweils 2D-Arrays, die das Pixel-Koordinatengitter zum Vektorfeld definieren.
    * *U*, *V*: jeweils 2D-Arrays (Dimensionen entsprechen (*len(x), len(y)*), die die Geschwindigkeitskomponenten in x- und y-Richtung enthalten.

Hinweise:

  * Sinnvollerweise kann zum Laden der Dateien direkt obige Funktion *loaddata()* verwendet werden.
  * Die Größe des zurückgegebenen Vektorfeldes bestimmt sich aus der Größe der Ausgangsbilder, der Größe der Interrogationsfenster sowie der Suchfenster.
  * Ein mit Nullen gefülltes Array lässt sich z.B. mit NumPys [*zeros()*](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html)-Funktion erzeugen.
  * Die quadratischen *Interrogations*fenster grenzen immer direkt aneinander. Ein zu einem Interrogationsfenster gehörendes *Such*fenster hat den gleichen Mittelpunkt wie das Interrogationsfenster.
  * Eine zweidimensionale Kreuzkorrelation lässt sich z.B. mit den Funktionen [*correlate2d()*](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.correlate2d.html) oder [*fftconvolve()*](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.fftconvolve.html) aus dem Paket *scipy.signal* berechnen.
  * Für eine leichtere Fehlersuche ist es sinnvoll, mit Testausgaben (oder -plots) zu überprüfen, ob Zwischenschritte den Erwartungen entsprechen.
  * Um aus einem Vektor mit x-Koordinaten und einem Vektor mit y-Koordinaten ein 2D-Gitter aufzuspannen (auf dem alle x- und y-Koordinaten kombiniert werden), kann die NumPy-Funktion [*meshgrid()*](https://docs.scipy.org/doc/numpy/reference/generated/numpy.meshgrid.html) verwendet werden.

    

In [6]:
%matplotlib widget
im1=im1/256
im2=im2/256
cross = np.array([[0,0,1,0],
                  [0,0,1,0],
                  [1,1,1,1],
                  [0,0,1,0]])
test_im = np.zeros((20, 20))
trans_x = 3
trans_y = 3
test_im[8+trans_x:12+trans_x, 8+trans_y:12+trans_y] = cross
plt.imshow(im1[200:220, 200:220])

x_start = 200
x_range = 20
y_start = 200
y_range = 20

corr = signal.correlate2d(im2, im1[x_start:x_start+x_range, y_start:y_start+y_range], mode='same')
#plt.imshow(corr)
a = np.asarray(corr==corr.max()).nonzero()
#plt.imshow(corr[380:400, 380:400])
plt.imshow(im2[a[0][0]-x_range//2:a[0][0]+x_range//2,a[1][0]-y_range//2:a[1][0]+y_range//2])
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
# Hier eigenen Code 
def piv(path1, path2, size_interr_window=20, size_search_window=None):
    im1, im2 = loaddata(path1, path2)
    if size_search_window == None:
        size_search_window = size_interr_window
    elif size_search_window < size_interr_window:
        raise ValueError("Search window must be equal or bigger than interrogation window.") 

    size_interr_lower = size_interr_window//2
    size_interr_upper = size_interr_lower if size_interr_window%2 == 0 else size_interr_lower+1 
    size_search_lower = size_search_window//2
    size_search_upper = size_search_lower if size_search_window%2 == 0 else size_search_lower+1 
    
    x_axis = np.arange(size_interr_window//2, im1.shape[0], size_interr_window)
    y_axis = np.arange(size_interr_window//2, im1.shape[1], size_interr_window)
    [x_grid, y_grid] = np.meshgrid(x_axis, y_axis)
    U = np.zeros(x_grid.shape)
    V = np.zeros(x_grid.shape)
    for i, x in enumerate(x_axis):
        for j, y in enumerate(y_axis):
            corr = signal.correlate2d(im2[x - size_search_lower:x + size_search_upper,
                                          y - size_search_lower:y + size_search_upper],
                                      im1[x - size_interr_lower:x + size_interr_upper,
                                          y - size_interr_lower:y + size_interr_upper],
                                      mode='same')
            max_x, max_y = np.asarray(corr==corr.max()).nonzero()
            U[i][j] = max_x - size_search_lower
            V[i][j] = max_y - size_search_lower
    return x_grid, y_grid, U, V
            
X, Y, U, V = piv(prefix + path_list[0][0], prefix + path_list[0][1])
print(U)

[[ -2.  -2.  -2.  -2.  -2.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.
    0.  -1.  -1.   0.  -1.  -1.   0.   0.   0.   0.   0.   0.]
 [ -2.  -2.  -2.  -2.  -2.  -2.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -2.  -1.  -2.  -2.  -2.  -2.  -1.  -1.  -1.  -1.  -1.  -1.  -1.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -2.  -2.  -2.  -2.  -2.  -2.  -2.  -1.  -1.  -1.  -1.  -1.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -2.  -2.  -2.  -2.  -2.  -2.  -2.  -2.  -1.  -1.  -1.  -1.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -2.  -2.  -2.  -2.  -2.  -2.  -2.  -2.  -1.  -1.  -1.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -2.  -3.  -2.  -3.  -3.  -2.  -2.  -2.  -2.  -1.  -1.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -2.  -3.  -3.  -3.  -3.  -3.  -3.  -3.  -2.  -1.  -1

In [32]:
a = np.array([[1,5],[3,4]])
b_x, b_y = np.asarray(a==a.max()).nonzero()
b_x, b_y

(array([0], dtype=int64), array([1], dtype=int64))

**4) Erstellen Sie mit Hilfe der *piv()*-Funktion für jeden Datensatz drei Darstellungen:**
  1. Einen Plot mit [Vektorpfeilen](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.quiver.html), der die Strömungsrichtungen der Partikel(-Gruppen) visualisiert. *Hinweis: Beachten Sie den optionalen Parameter "angles" für eine korrekte Darstellung.*
  2. Eine [Stromlinienvisualisierung](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.streamplot.html).
  3. Einen [Konturplot](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.contourf.html), der den Betrag der Geschwindigkeitsverteilung abbildet.

**Variieren Sie die Parameter der *piv()*-Funktion sowie der jeweiligen Plot-Funktion, um eine möglichst aussagekräftige Abbildung zu erhalten. Hinterlegen Sie den Plots zur besseren Übersicht eines der jeweiligen Partikelbilder.**

In [ ]:
# Hier eigenen Code schreiben und ggf. weitere Zellen hinzufügen ...


**5) Zusatzaufgaben (freiwillig):**
  * Fügen Sie einen optionalen Parameter "overlap" – und die entsprechende Implementierung – für überlappende Interrogationsfenster hinzu.
  * Aktuell können nur Geschwindigkeits-Vektoren mit Pixelgenauigkeit gefunden werden. Überlegen Sie, ob bzw. wie genauere Werte berechnet werden könnten.
  * Wie könnte das Ergebnis verbessert werden, die auftretenden Geschwindigkeiten stark variieren?